In [1]:
import os
os.environ["http_proxy"] = "http://127.0.0.1:8889"
os.environ["https_proxy"] = "http://127.0.0.1:8889"

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset, Dataset, DatasetDict

/home/dean/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ds = Dataset.load_from_disk("datasets/nlpcc_2017")
ds

Dataset({
    features: ['title', 'content'],
    num_rows: 5000
})

In [4]:
ds = ds.train_test_split(test_size=0.1)
ds

DatasetDict({
    train: Dataset({
        features: ['title', 'content'],
        num_rows: 4500
    })
    test: Dataset({
        features: ['title', 'content'],
        num_rows: 500
    })
})

In [5]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/glm-large-chinese",trust_remote_code=True)

def process_func(examples):
    contents = ["摘要生成：\n" + c + tokenizer.mask_token for c in examples["content"]]
    inputs = tokenizer(contents, truncation=True, max_length=384, padding="max_length", return_tensors="pt")
    inputs = tokenizer.build_inputs_for_generation(inputs, max_gen_length=64, targets=examples["title"], padding=True)
    return inputs

In [6]:
tokenized_ds = ds.map(process_func, batched=True, batch_size=10, remove_columns=["content", "title"])
tokenized_ds

Map: 100%|██████████| 500/500 [00:17<00:00, 28.77 examples/s] 


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'position_ids', 'attention_mask', 'labels'],
        num_rows: 4500
    })
    test: Dataset({
        features: ['input_ids', 'position_ids', 'attention_mask', 'labels'],
        num_rows: 500
    })
})

In [7]:
tokenizer.decode(tokenized_ds["train"][0]["input_ids"])

'[CLS] 摘要生成: 东亚杯首轮,国足0-2不敌韩国。前恒大主帅、曾在中国执教10多年的韩国教练李章洙表示,这场比赛的球员一点都没有责任心、使命感,没有赢球的欲望。李章洙首先表示:“看了东亚杯第一场中韩之战,我真的没想到,以这支韩国队球员的构成来看,中国球员竟然能把比赛踢成这样,真的太让我失望了,不知道广大的中国球迷怎么想。”接着,李章洙表示:“这场比赛的球员一点都没有责任心,使命感,没有赢球的欲望。比赛进行到45分钟以后,已经完全放弃了比赛,完全没有了取胜的欲望。”“当然了,虽然进行了这样的比赛,也可以把他们称作职业球员。在中国,他们享受着球星级的待遇,他们的责任感也必须要跟上去。难道不是这样的吗?”“在这方面,这些人做的确实有一些不足。不管有什么样的客观因素,至少这场比赛的内容,他们应该感到羞耻。”对于韩国队,李章洙表示:“韩国队在这次东亚杯招入的球员,基本都是年轻的球员,在欧洲效力的球员都没有回来。像进球的李宗浩,都是首次进入国家队,但他有着对比赛必胜的欲望和态度。” [MASK] <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <

In [8]:
model = AutoModelForSeq2SeqLM.from_pretrained("THUDM/glm-large-chinese", trust_remote_code=True)

In [9]:
args = Seq2SeqTrainingArguments(
    output_dir="./models/summarization_glm",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_strategy="steps",
    logging_steps=25,
    num_train_epochs=1,
)

In [10]:
trainer = Seq2SeqTrainer(
    args=args,
    model=model,
    train_dataset=tokenized_ds["train"],
    tokenizer=tokenizer,
)

In [12]:
import torch
torch.cuda.empty_cache()
trainer.train()

Step,Training Loss
25,1.800700
50,1.669400
75,1.645300
100,1.583900
125,1.518800


TrainOutput(global_step=140, training_loss=1.6298928260803223, metrics={'train_runtime': 603.7232, 'train_samples_per_second': 7.454, 'train_steps_per_second': 0.232, 'total_flos': 4485750309519360.0, 'train_loss': 1.6298928260803223, 'epoch': 1.0})

In [17]:
input_text = ds["test"][0]["content"]
inputs = tokenizer("摘要生成：\n" + input_text + tokenizer.mask_token, return_tensors="pt")
inputs = tokenizer.build_inputs_for_generation(inputs, max_gen_length=64)
inputs = inputs.to("cuda")
output = model.generate(**inputs, max_new_tokens=64, eos_token_id=tokenizer.eop_token_id)
tokenizer.decode(output[0].tolist())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50007 for open-end generation.


'[CLS] 摘要生成: 5月26日,火灾现场只剩铁架子。平顶山“5·25”火灾事故发生后,习近平立即作出重要指示,要求河南省和有关部门全力救治受伤人员,妥善做好遇难者善后和家属安抚工作,并查明事故原因,依法追究事故责任。李克强作出批示,要求抓紧全力抢救伤员,做好伤亡人员家属安抚等工作。“5.25”火灾事故基本情况5月25日19时33分,平顶山市鲁山县城西琴台办事处三里河村康乐园老年公寓发生火灾。接到报警后,19时49分,公安消防部队赶到现场,迅速开展灭火救援工作;20时22分许,火势基本扑灭,但着火房屋已经坍塌。现场救援人员对火灾区域进行反复搜救排查。截至26日凌晨6时,火灾现场清理完毕,共搜救出44人,其中:死亡38人、重伤2人、轻伤4人。伤者已全部送医院救治。鲁山县康乐园老年公寓基本情况2010年12月,平顶山市民政局批准成立鲁山县康乐园老年公寓,该项目位于县城鲁平大道西段,占地面积30亩。此次火灾波及房屋建筑面积600平方米,过火区域有床位51个,火灾当晚有44人居住,除死伤人员外,其余7人已全部找到下落。目前,受伤人员己全部送治,相关责任人已经被控制,事故原因正在调查中。 [MASK] <|endoftext|> <|startofpiece|> 平顶山鲁山康乐园老年公寓发生火灾,已搜救出44人,其中死亡38人、重伤2人、轻伤4人。 <|endofpiece|>'

In [42]:
import torch

model = model.eval()


def predict_test():
    cnt = 0
    predict = []
    with torch.inference_mode():
        for d in ds["test"]:
            cnt +=1
            inputs = tokenizer("摘要生成：\n" + d["content"] + tokenizer.mask_token, return_tensors="pt")
            inputs = tokenizer.build_inputs_for_generation(inputs, max_gen_length=64)
            inputs = inputs.to("cuda")
            output = model.generate(**inputs, max_new_tokens=64, eos_token_id=tokenizer.eop_token_id)
            predict.append(tokenizer.decode(output[0].tolist()).split("<|startofpiece|>")[1].replace("<|endofpiece|>", "").strip())
            print("curID:", len(predict))
            if cnt == 10:
                break
    return predict

In [43]:
result = predict_test()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50007 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50007 for open-end generation.


curID: 1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50007 for open-end generation.


curID: 2


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50007 for open-end generation.


...

curID: 7


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50007 for open-end generation.


curID: 8


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50007 for open-end generation.


curID: 9
curID: 10


In [44]:
result

['平顶山鲁山康乐园老年公寓发生火灾,已搜救出44人,其中死亡38人、重伤2人、轻伤4人。',
 '武汉:女司机抱怨越野车多次按喇叭,遭司机暴打近5分钟;事发鲁磨路公交车站附近,女司机下车抱怨时遭暴打。',
 '联通将iPhone4s裸机价格降至1499元,这是苹果官方第二次降价,此前iPhone4s已下架。',
 '上海迪士尼乐园拥有全球迪士尼乐园中最高、最大、最具互动性的城堡,是迪士尼城堡中首个汇集所有迪士尼公主的城堡。',
 '日本首相安倍晋三倡导“提早办公”模式,预计中央省厅的加班时间将增加,措施推广伊始就面临“黄灯”阻碍。',
 '朝鲜称将进行第四次核试验,美称其决议是“严重的政治挑衅”,并威胁称将进行核试验。',
 '江苏省发布高温橙色预警:预计未来24小时内南京、常州、无锡、镇江、扬州中南部最高气温将升至37°C以上,苏州、南通、盐城、淮安、宿迁、泰州、连云港和扬州北部最高气温将达36°C左右。...',
 '西安:男子300万豪车停路边,突然听到有人划车,车主报警,小孩共划伤9辆车。',
 '组图:加州近海出现一道奇观,成千上万凤尾鱼聚集在一起,速度非常快,吸引上千只海鸥前来捕食。',
 '皇马公布中国赛大名单,C罗领衔贝尔本泽马,J罗、摩德里奇等均入选']

In [53]:
from rouge_chinese import Rouge
rouge = Rouge()

decoded_preds = []
for i in range(10):
    decoded_preds.append(' '.join(result[i]))


decoded_labels = []
for i in range(10):
    decoded_labels.append(' '.join(ds["test"][i]["title"]))

    
scores = rouge.get_scores(decoded_preds, decoded_labels, avg=True)
{
    "rouge-1": scores["rouge-1"]["f"],
    "rouge-2": scores["rouge-2"]["f"],
    "rouge-l": scores["rouge-l"]["f"],
}

{'rouge-1': 0.5067325073377883,
 'rouge-2': 0.30063484821263986,
 'rouge-l': 0.4130530102195234}